In [1]:
from IPython.display import HTML, Javascript, display
import IPython
from ipywidgets import widgets

hide_all_code_button = HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code for this notebook."></form>''')
display(hide_all_code_button)


def run_below(ev):
    display(Javascript('IPython.notebook.execute_cells([IPython.notebook.get_selected_index()+1])'))

run_code_below_button = widgets.Button(description="Run code")
run_code_below_button.on_click(run_below)

# # So far we enter the code button cells manually - we need to find a way to automate this.
# CELLS_TO_RUN = [2, # Cell that includes the dependencies for the notebook (python libraries)
#                 3  # Cell that includes the helper functions
#                ]
# CODE_BUTTON_CELL_IDXS = [6]
# # Generate code button cells
# Javascript(f'IPython.notebook.execute_cells({CELLS_TO_RUN + CODE_BUTTON_CELL_IDXS})')

<IPython.core.display.Javascript object>

In [5]:
# Basic dependencies
import math

# Numerical and dataframe dependencies
import numpy as np
import pandas as pd

# Plotting dependencies
import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

# Deep Learning 
import tensorflow as tf

%matplotlib inline

In [2]:
# Helper functions


# Intro to AI Notebook

<img src="images/IntroToAIOpeningImage.jpeg" width="600">

Welcome to the Jupyter notebook! This is the classic environment for a data scientist and Machine Learning Engineer. The first code cell that you ran hides all of the code we will be running today, so you don't have to worry about it. But, if you would like to take a look then you can toggle the code on and off at any time!

The intention of this notebook is to give you an interactive and friendly introduction to AI and Machine Learning by stepping through a classic data scientist.

<img src="images/DataScienceLifeCycle.png" width="400" title="Data Science Workflow">

A classic Machine Learning workflow is made up of the following general steps

- Define Problem
- Specify Inputs & Outputs
- Exploratory data analysis
- Data Collection
- Data Preprocessing
- Data Cleaning
- Visualization
- Model Design, Training, and Offline Evaluation
- Model Deployment, Online Evaluation, and Monitoring
- Model Maintenance, Diagnosis, and Retraining

<img src="images/MLWorkflow.png" width="800" title="ML Workflow">

<img src="images/MLWorkflowLowLevel.png" title="Low Level ML Workflow">



In [7]:
display(run_code_below_button)

Button(description='Run code', style=ButtonStyle())

In [3]:
print('Hello world')

Hello world


## Test Python

In [ ]:
print('Hello world!')

In [ ]:
import numpy as np

print(np.__version__)

In [ ]:
import pandas as pd

print(pd.__version__)

In [ ]:
## Nice image slider: https://sanjayasubedi.com.np/deeplearning/tensorflow-2-first-neural-network-for-fashion-mnist/
from ipywidgets import interact, widgets
img_idx_slider = widgets.IntSlider(value=0, min=0, max=len(x_test)-1, description="Image index")

@interact(i=img_idx_slider)
def visualize_prediction(i=0):
    fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(x_test[i], cmap=plt.cm.binary)
    ax1.set_title(f"lbl: {class_names[y_test[i]]}")
    ax1.set_xlabel(f"pred: {class_names[preds[i]]}")


    ax2.bar(x=[class_names[i] for i in range(10)], height=probs[i]*100)
    plt.xticks(rotation=90)